# Sprint 機械学習スクラッチ入門

【目的】機械学習スクラッチの準備をする

【概要】今後の機械学習スクラッチ課題で作成するモデルを、scikit-learnを用いて一度動かしておきます。これまでの復習を兼ねたスクラッチ課題の準備です。

## 【問題1】train_test_splitのスクラッチ
スクラッチの練習として、scikit-learnのtrain_test_splitを自作してみます。以下の雛形をベースとして関数を完成させてください。<br>
なお、作成した関数がscikit-learnのtrain_test_splitと同じ動作をしているか必ず確認をするようにしましょう。

In [1]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [2]:
def scratch_train_test_split(X, y, train_size=0.8, random_state=None, shuffle=True):
    """
    検証データを分割する
    Parameters
    --------------
    X : 次の形のndarray, shape(n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape(n_samples, )
      正解値
    train_size : float (0 <train_saize<1)
      何割をtrainとするか指定
    random_state : int
      分割の前にデータをシャッフルする場合の制御値
    shuffle : boolean
      シャッフルするかどうか
    
    Returns
    --------------
    X_train : 次の形のndarray, shape(n_samples, n_features)
      訓練データ
    X_test : 次の形のndarray, shape(n_samples, n_features)
      検証データ
    y_train : 次の形のndarray, shape(n_samples, )
      訓練データの正解値
    y_test : 次の形のndarray, shape(n_samples, )
      検証データの正解値
    """
    
    # データのサイズのみチェック
    # yがrangeやlistで来ても対応できるようにndarrayに変換
    y = np.array(list(y))
    assert X.shape[0] == y.shape[0], "size error"
    
    # シャッフル用のNo.生成
    number_ndarray = np.arange(y.shape[0])
    
    # シード値を入れる
    if random_state != None:
        random.seed(random_state)
    
    if shuffle == True:
        # No.をシャッフル
        random.shuffle(number_ndarray)
        
    # No.とXとyを結合
    number_ndarray = number_ndarray.reshape(X.shape[0], 1)
    y = y.reshape(X.shape[0], 1)
    
    all_ndarray = np.concatenate((number_ndarray, X, y), axis=1)
    
    # No.(0列目を基準に)でソート
    index = np.argsort(all_ndarray[:, 0])
    all_ndarray_sort = all_ndarray[index, :]
    
    # 分割
    split_i = round(all_ndarray_sort.shape[0]*train_size)
    #X_train, X_test, y_train, y_test
    return  (all_ndarray_sort[:split_i, 1:-1], all_ndarray_sort[split_i:, 1:-1],
    all_ndarray_sort[:split_i, -1:].reshape(split_i,), 
    all_ndarray_sort[split_i:, -1:].reshape(all_ndarray_sort.shape[0] - split_i))

In [3]:
# X, y = np.arange(20).reshape((10, 2)), range(10)
X = np.arange(20).reshape((10, 2))
y = np.arange(10).reshape((10, ))

In [4]:
# train_test_splitの挙動を確認
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=1, shuffle=False)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
# rain_test_splitの戻り値を確認
print("---X_train---")
print(X_train)
print("---X_test---")
print(X_test)
print("---y_train---")
print(y_train)
print("---y_test---")
print(y_test)

---X_train---
[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]]
---X_test---
[[14 15]
 [16 17]
 [18 19]]
---y_train---
[0 1 2 3 4 5 6]
---y_test---
[7 8 9]


In [6]:
print(X)
print(y)
X_train, X_test, y_train, y_test = scratch_train_test_split(X, y, train_size=0.7, random_state=1)
print("---X_train---")
print(X_train)
print("---X_test---")
print(X_test)
print("---y_train---")
print(y_train)
print("---y_test---")
print(y_test)

[[ 0  1]
 [ 2  3]
 [ 4  5]
 [ 6  7]
 [ 8  9]
 [10 11]
 [12 13]
 [14 15]
 [16 17]
 [18 19]]
[0 1 2 3 4 5 6 7 8 9]
---X_train---
[[12 13]
 [16 17]
 [18 19]
 [10 11]
 [14 15]
 [ 8  9]
 [ 0  1]]
---X_test---
[[6 7]
 [2 3]
 [4 5]]
---y_train---
[6 8 9 5 7 4 0]
---y_test---
[3 1 2]


train_test_splitのスクラッチをした。random_stateのシード値で結果が固定されること、shuffleがTrueのとき、データがシャッフルされることも確認した。

## 【問題2】 分類問題を解くコードの作成
3種類の手法（ロジスティック回帰、SVM、決定木）で<br>
3種類のデータセット（irisのvirgicolorとvirginica特徴量は全て、シンプルデータセット1、シンプルデータセット2）を学習・推定するコードを作成してください。

In [7]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import pandas as pd

### Iris

In [8]:
iris_data = datasets.load_iris()
input_data = iris_data.data
X = pd.DataFrame(data=input_data, columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])
correct = iris_data.target
y = pd.DataFrame(data=correct, columns=["Species"])
df = pd.concat([X, y], axis=1)
df.drop(df.index[df["Species"]==0], inplace=True)
#df

In [9]:
# スクラッチしたtrain_test_splitでtrainとvalidationに分ける
X_train, X_val, y_train, y_val = scratch_train_test_split(df.iloc[:, :-1].values, df.iloc[:, -1:].values, train_size=0.75)

In [10]:
print("X_train",X_train.shape)
print("X_val",X_val.shape)
print("y_train",y_train.shape)
print("y_val",y_val.shape)

X_train (75, 4)
X_val (25, 4)
y_train (75,)
y_val (25,)


In [11]:
# 標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)

In [12]:
# 学習と推定:ロジスティック回帰(勾配降下法を使用)
from sklearn.linear_model import SGDClassifier
sgdc_model = SGDClassifier(loss="log")
sgdc_model.fit(X_train_std, y_train)
y_val_pred = sgdc_model.predict(X_val_std)
y_val_pred

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([2., 1., 2., 1., 1., 1., 1., 2., 2., 2., 2., 1., 1., 2., 2., 1., 1.,
       1., 1., 2., 1., 2., 1., 1., 1.])

In [13]:
# 学習と推定:SVM
from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(X_train_std, y_train)
y_val_pred = svc_model.predict(X_val_std)
y_val_pred

array([2., 1., 2., 1., 1., 1., 1., 2., 2., 2., 2., 1., 1., 2., 2., 1., 1.,
       1., 1., 2., 1., 2., 1., 1., 1.])

In [14]:
# 学習と推定:決定木
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_std, y_train)
y_val_pred = dt_model.predict(X_val_std)
y_val_pred

array([2., 1., 2., 1., 1., 1., 1., 2., 2., 2., 2., 1., 1., 2., 2., 1., 1.,
       1., 1., 2., 1., 2., 1., 1., 1.])

### シンプルデータセット1

In [15]:
# シンプルデータセット1作成
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index]
y = y[random_index]

In [16]:
# スクラッチしたtrain_test_splitでtrainとvalidationに分ける
X_train, X_val, y_train, y_val = scratch_train_test_split(X, y, train_size=0.75)

In [17]:
# 標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)

In [18]:
# 学習と推定:ロジスティック回帰(勾配降下法を使用)
#from sklearn.linear_model import SGDClassifier
sgdc_model = SGDClassifier(loss="log")
sgdc_model.fit(X_train_std, y_train)
y_val_pred = sgdc_model.predict(X_val_std)
y_val_pred

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([-1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
       -1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,
       -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
        1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,
        1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
        1.,  1.,  1., -1.,  1.,  1., -1., -1.])

In [19]:
# 学習と推定:SVM
#from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(X_train_std, y_train)
y_val_pred = svc_model.predict(X_val_std)
y_val_pred

array([-1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
       -1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,
       -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
        1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,
        1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
        1.,  1.,  1., -1.,  1.,  1., -1., -1.])

In [20]:
# 学習と推定:決定木
#from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_std, y_train)
y_val_pred = dt_model.predict(X_val_std)
y_val_pred

array([-1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,
       -1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,
       -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1., -1.,
        1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,
       -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,
        1.,  1.,  1., -1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,
        1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,
        1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1., -1., -1.,
        1.,  1.,  1., -1.,  1.,  1., -1., -1.])

### シンプルデータセット2

In [21]:
# シンプルデータセット2作成
X = np.array([[-0.44699 , -2.8073  ],[-1.4621  , -2.4586  ],
       [ 0.10645 ,  1.9242  ],[-3.5944  , -4.0112  ],
       [-0.9888  ,  4.5718  ],[-3.1625  , -3.9606  ],
       [ 0.56421 ,  0.72888 ],[-0.60216 ,  8.4636  ],
       [-0.61251 , -0.75345 ],[-0.73535 , -2.2718  ],
       [-0.80647 , -2.2135  ],[ 0.86291 ,  2.3946  ],
       [-3.1108  ,  0.15394 ],[-2.9362  ,  2.5462  ],
       [-0.57242 , -2.9915  ],[ 1.4771  ,  3.4896  ],
       [ 0.58619 ,  0.37158 ],[ 0.6017  ,  4.3439  ],
       [-2.1086  ,  8.3428  ],[-4.1013  , -4.353   ],
       [-1.9948  , -1.3927  ],[ 0.35084 , -0.031994],
       [ 0.96765 ,  7.8929  ],[-1.281   , 15.6824  ],
       [ 0.96765 , 10.083   ],[ 1.3763  ,  1.3347  ],
       [-2.234   , -2.5323  ],[-2.9452  , -1.8219  ],
       [ 0.14654 , -0.28733 ],[ 0.5461  ,  5.8245  ],
       [-0.65259 ,  9.3444  ],[ 0.59912 ,  5.3524  ],
       [ 0.50214 , -0.31818 ],[-3.0603  , -3.6461  ],
       [-6.6797  ,  0.67661 ],[-2.353   , -0.72261 ],
       [ 1.1319  ,  2.4023  ],[-0.12243 ,  9.0162  ],
       [-2.5677  , 13.1779  ],[ 0.057313,  5.4681  ]])
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [22]:
# スクラッチしたtrain_test_splitでtrainとvalidationに分ける
X_train, X_val, y_train, y_val = scratch_train_test_split(X, y, train_size=0.75)

In [23]:
# 標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)

In [24]:
# 学習と推定:ロジスティック回帰(勾配降下法を使用)
#from sklearn.linear_model import SGDClassifier
sgdc_model = SGDClassifier(loss="log")
sgdc_model.fit(X_train_std, y_train)
y_val_pred = sgdc_model.predict(X_val_std)
y_val_pred

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
# 学習と推定:SVM
#from sklearn.svm import SVC
svc_model = SVC()
svc_model.fit(X_train_std, y_train)
y_val_pred = svc_model.predict(X_val_std)
y_val_pred

array([0., 1., 0., 0., 0., 0., 1., 0., 0., 0.])

In [26]:
# 学習と推定:決定木
#from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_std, y_train)
y_val_pred = dt_model.predict(X_val_std)
y_val_pred

array([0., 0., 1., 1., 1., 0., 1., 1., 0., 0.])

## 【問題3】 回帰問題を解くコードの作成
線形回帰(SGDRegressor)でHouse Pricesデータセット(GrLivAreaとYearBuilt)を学習・推定するコードを作成してください。

In [27]:
df = pd.read_csv("train.csv")
index_array = df.columns
index_list = index_array.tolist()
index_list.remove("GrLivArea")
index_list.remove("YearBuilt")
index_list.remove("SalePrice")
df.drop(columns=index_list, inplace=True)

In [28]:
# スクラッチしたtrain_test_splitでtrainとvalidationに分ける
X_train, X_val, y_train, y_val = scratch_train_test_split(df.iloc[:, :-1].values, df.iloc[:, -1:].values, train_size=0.75)

In [29]:
# 標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_val_std = scaler.transform(X_val)

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [30]:
# 学習と推定:線形回帰（勾配降下法使用）
from sklearn.linear_model import SGDRegressor
sgd_model = SGDRegressor()
sgd_model.fit(X_train_std, y_train)
y_val_pred = sgd_model.predict(X_val_std)
y_val_pred

/Users/arisa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


array([273097.57313455, 110629.42194599, 131631.1378898 , 105502.78054784,
       222323.35646856, 154298.45093217, 125669.06742418, 347288.30662581,
       238925.56630035, 216935.21904042, 207537.63798332, 238713.85403717,
       120640.58347151, 147400.61239168, 148684.99969363, 232456.02758572,
       103610.07381385, 124642.58450215, 198953.92967254, 170247.94504023,
       235575.89525655, 132292.70629999, 157216.35693877, 160936.20353081,
       246087.98034528,  67538.30338425, 178436.84200719, 191414.28321798,
       202537.7672132 , 166565.9500541 , 272232.632972  , 243124.78013446,
       111462.66945153, 348795.44043933, 108841.03160637, 191809.99247431,
       272096.62392063, 174553.27109374,  98806.51793468, 167518.91132626,
       286589.15371887, 163143.65527294,  90028.67629489, 192073.67006623,
       138758.77980327, 122962.48794387, 134376.46688854, 101057.08231909,
       145176.86536479, 122926.8178999 , 243834.33648642, 299483.44911954,
        94908.8332984 , 1